In [1]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [2]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")

from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet

In [3]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [4]:
DATA_ROOT_PATH = '../input/global-wheat-detection/test'

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [5]:
dataset = DatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [6]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

net = load_net('../input/wheat-effdet5-fold0-best-checkpoint/fold0-best-all-states.bin')

In [7]:
import ast

DATAPATH = '../input/global-wheat-detection'
SPLIT = 0.2

df = pd.read_csv(DATAPATH + '/train.csv')
df.bbox = df.bbox.apply(ast.literal_eval)


marking = pd.read_csv('../input/global-wheat-detection/train.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="bboxes")

def train_test_split(dataFrame,split):
    len_tot = len(dataFrame)
    val_len = int(split*len_tot)
    train_len = len_tot-val_len
    train_data,val_data = dataFrame.iloc[:train_len][:],dataFrame.iloc[train_len:][:]
    return train_data,val_data

train_data_df,val_data_df = train_test_split(df,SPLIT)

marking.head()

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0


In [8]:
def make_predictions(images, score_threshold=0.5):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
            })
    return [predictions]

def run_wbf(predictions, image_index, image_size=512, iou_thr=0.44, skip_box_thr=0.43, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [np.ones(prediction[image_index]['scores'].shape[0]).tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [9]:
'''import matplotlib.pyplot as plt

for j, (images, image_ids) in enumerate(data_loader):
    break

predictions = make_predictions(images)

i = 0

sample = images[i].permute(1,2,0).cpu().numpy()
print(image_ids[i])
boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=511)

records = marking[marking['image_id'] == image_ids[i]]
#print(records)
targets = records[['x', 'y', 'w', 'h']].values
targets[:, 2] = targets[:, 0] + targets[:, 2]
targets[:, 3] = targets[:, 1] + targets[:, 3]
#print(targets)

#print(boxes)
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

#X1,Y1,X2,Y2
for box in boxes:
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)
    
#for box in targets:
 #   cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (1, 0, 1), 1)
    
ax.set_axis_off()
ax.imshow(sample);

print(np.average(scores))'''

"import matplotlib.pyplot as plt\n\nfor j, (images, image_ids) in enumerate(data_loader):\n    break\n\npredictions = make_predictions(images)\n\ni = 0\n\nsample = images[i].permute(1,2,0).cpu().numpy()\nprint(image_ids[i])\nboxes, scores, labels = run_wbf(predictions, image_index=i)\nboxes = boxes.astype(np.int32).clip(min=0, max=511)\n\nrecords = marking[marking['image_id'] == image_ids[i]]\n#print(records)\ntargets = records[['x', 'y', 'w', 'h']].values\ntargets[:, 2] = targets[:, 0] + targets[:, 2]\ntargets[:, 3] = targets[:, 1] + targets[:, 3]\n#print(targets)\n\n#print(boxes)\nfig, ax = plt.subplots(1, 1, figsize=(16, 8))\n\n#X1,Y1,X2,Y2\nfor box in boxes:\n    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)\n    \n#for box in targets:\n #   cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (1, 0, 1), 1)\n    \nax.set_axis_off()\nax.imshow(sample);\n\nprint(np.average(scores))"

In [10]:
'''b1 = targets
b2 = boxes
sample = images[i].permute(1,2,0).cpu().numpy()

#scores = outputs[i]['scores'].detach().numpy()
# print(scores)
#b2 = b2[scores >= 0.5].astype(np.int32)

b2 = [x for _ , x in sorted(zip(scores,b2), key=lambda x: x[0], reverse = True)]
b2 = b2[:(b1.shape[0])]
b_good = []
#print(len(b2))
numerator = 0
denominator = 0

for bb2 in b2:
    for bb1 in b1:
        #print(bb1)
        #print(bb2)
        o = bb_intersection_over_union(bb1, bb2)
        #print(o)
        num = o[0]
        dem = o[1]
        if num>0:# and num/dem>0.25:
            numerator += num
            denominator +=dem
            b_good.append(bb2)
if denominator!=0:
    iou = numerator/denominator
print(iou)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
#print(b1)


for box in b1:
    cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
                  (255, 0, 0), 1)
#print(b_good)
for box in b_good:
    cv2.rectangle(sample,(int(box[0]), int(box[1])), (int(box[2]), int(box[3])),
                  (0, 0, 255), 1)
ax.set_axis_off()
ax.imshow(sample)
    
    
#print(val_iou)'''

"b1 = targets\nb2 = boxes\nsample = images[i].permute(1,2,0).cpu().numpy()\n\n#scores = outputs[i]['scores'].detach().numpy()\n# print(scores)\n#b2 = b2[scores >= 0.5].astype(np.int32)\n\nb2 = [x for _ , x in sorted(zip(scores,b2), key=lambda x: x[0], reverse = True)]\nb2 = b2[:(b1.shape[0])]\nb_good = []\n#print(len(b2))\nnumerator = 0\ndenominator = 0\n\nfor bb2 in b2:\n    for bb1 in b1:\n        #print(bb1)\n        #print(bb2)\n        o = bb_intersection_over_union(bb1, bb2)\n        #print(o)\n        num = o[0]\n        dem = o[1]\n        if num>0:# and num/dem>0.25:\n            numerator += num\n            denominator +=dem\n            b_good.append(bb2)\nif denominator!=0:\n    iou = numerator/denominator\nprint(iou)\n\nfig, ax = plt.subplots(1, 1, figsize=(16, 8))\n#print(b1)\n\n\nfor box in b1:\n    cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])),\n                  (255, 0, 0), 1)\n#print(b_good)\nfor box in b_good:\n    cv2.rectangle(sampl

In [11]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return interArea, float(boxAArea + boxBArea - interArea)

In [12]:
'''results = []
avg_score = []
for images, image_ids in data_loader:
    predictions = make_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*2).astype(np.int32).clip(min=0, max=1023)
        
        boxes = boxes.astype(np.int32).clip(min=0, max=511)
        image_id = image_ids[i]
        records = marking[marking['image_id'] == image_ids[i]]
        #print(records)
        targets = records[['x', 'y', 'w', 'h']].values
        targets[:, 2] = targets[:, 0] + targets[:, 2]
        targets[:, 3] = targets[:, 1] + targets[:, 3]
        #print(targets)

        #print(boxes)
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))

        #X1,Y1,X2,Y2
        for box in boxes:
            cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)

        #for box in targets:
         #   cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (1, 0, 1), 1)

        ax.set_axis_off()
        ax.imshow(sample);

        print("scores = ", np.average(scores))
        
        b1 = targets
        b2 = boxes
        sample = images[i].permute(1,2,0).cpu().numpy()

        #scores = outputs[i]['scores'].detach().numpy()
        # print(scores)
        #b2 = b2[scores >= 0.5].astype(np.int32)

        b2 = [x for _ , x in sorted(zip(scores,b2), key=lambda x: x[0], reverse = True)]
        b2 = b2[:(b1.shape[0])]
        b_good = []
        #print(len(b2))
        numerator = 0
        denominator = 0

        for bb2 in b2:
            for bb1 in b1:
                #print(bb1)
                #print(bb2)
                o = bb_intersection_over_union(bb1, bb2)
                #print(o)
                num = o[0]
                dem = o[1]
                if num>0:# and num/dem>0.25:
                    numerator += num
                    denominator +=dem
                    b_good.append(bb2)
        if denominator!=0:
            iou = numerator/denominator
        print("iou = ", iou, '\n')
        
        
        
        avg_score.append(scores)
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        results.append(result)'''

'results = []\navg_score = []\nfor images, image_ids in data_loader:\n    predictions = make_predictions(images)\n    for i, image in enumerate(images):\n        boxes, scores, labels = run_wbf(predictions, image_index=i)\n        boxes = (boxes*2).astype(np.int32).clip(min=0, max=1023)\n        \n        boxes = boxes.astype(np.int32).clip(min=0, max=511)\n        image_id = image_ids[i]\n        records = marking[marking[\'image_id\'] == image_ids[i]]\n        #print(records)\n        targets = records[[\'x\', \'y\', \'w\', \'h\']].values\n        targets[:, 2] = targets[:, 0] + targets[:, 2]\n        targets[:, 3] = targets[:, 1] + targets[:, 3]\n        #print(targets)\n\n        #print(boxes)\n        fig, ax = plt.subplots(1, 1, figsize=(16, 8))\n\n        #X1,Y1,X2,Y2\n        for box in boxes:\n            cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 1)\n\n        #for box in targets:\n         #   cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(b

In [13]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [14]:
results = []
avg_score = []
for images, image_ids in data_loader:
    predictions = make_predictions(images)
    for i, image in enumerate(images):
        boxes, scores, labels = run_wbf(predictions, image_index=i)
        boxes = (boxes*2).astype(np.int32).clip(min=0, max=1023)
        image_id = image_ids[i]

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        avg_score.append(scores)
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }
        if len(result['image_id'])!=9:
            continue
        else:
            results.append(result)

/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [15]:
fs = []
for x in avg_score:
    if np.isnan(np.mean(x)):
        continue
    else:
        fs.append(np.mean(x))

print(np.mean(fs))

0.762193871025876


In [16]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df

,image_id,PredictionString
0,796707dd7,0.9153 708 824 110 102 0.8817 893 330 114 94 0...
1,2fd875eaa,0.9358 460 499 79 133 0.9146 531 869 111 118 0...
2,cc3532ff6,0.9519 772 829 162 161 0.9396 909 123 112 95 0...
3,53f253011,0.9349 142 920 110 103 0.9172 233 840 115 95 0...
4,f5a1f0358,0.9284 542 271 110 114 0.9073 601 727 104 90 0...
5,51f1be19e,0.8388 834 269 140 204 0.8094 610 84 156 168 0...
6,aac893a91,0.8878 570 539 104 179 0.8570 240 81 132 148 0...
7,cb8d261a3,0.9352 309 158 107 212 0.9253 19 866 82 142 0....
8,51b3e36ab,0.8983 836 446 187 149 0.8931 496 361 314 128 ...
9,348a992bb,0.9160 736 223 139 86 0.8631 114 929 88 72 0.8...


### Thank you for reading my kernel!

If you didn't read training and WBF kernels:

- [[Training] EfficientDet](https://www.kaggle.com/shonenkov/training-efficientdet)
- [WBF approach for ensemble](https://www.kaggle.com/shonenkov/wbf-approach-for-ensemble)


Just recently I have started publishing my works, if you like this format of notebooks I would like continue to make kernels.